<a href="https://colab.research.google.com/github/03aquamarine/YuJin-s/blob/main/0719Wed_%EA%B0%9C%EC%9D%B8%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B802_%EC%84%B1%EA%B7%A0%EA%B4%80%EB%8C%80_%EB%85%B8%EC%9C%A0%EC%A7%84_%EB%B6%88%EB%9F%89_%EC%A0%9C%ED%92%88_%EC%9E%90%EB%8F%99_%ED%83%90%EC%A7%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. data 불러오기
2. labeling
3. 전처리
4. 학습
5. predict

# 1. Load data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Data 일괄 불러오기
import os , glob

good_data = glob.glob('/content/drive/MyDrive/grid/grid/train/good/*')
anomaly_data = glob.glob('/content/drive/MyDrive/grid/grid/test/*/*.png')

In [ ]:
good_data
anomaly_data

# 2. Data labeling & preprocessing


In [ ]:
import numpy as np
from PIL import Image

# 1. resize
# 2. type( RGB로)
# 3. 픽셀값 0~1 normalize (cf.생성 -1 ~ 1)
# 4. numpy

x = []
y = []
data_folder = [good_data,anomaly_data]

for idx,folder in enumerate(data_folder):
  for file_path in folder:
      # 1. resize
      image = Image.open(file_path)
      image = image.resize((128,128))

      # 2. type to RGB
      image = image.convert("RGB")

      # 3. normalize pixel values (0~1)
      image = np.array(image) / 255

      # 4. store images as a numpy array
      x.append(image)
      y.append(idx)

x = np.array(x)
y = np.array(y)

In [ ]:
from sklearn.model_selection import train_test_split

# training sets와 testing sets로 data 분할
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Shuffle the training data
shuffle_indices = np.arange(len(x_train))
np.random.shuffle(shuffle_indices)
x_train = x_train[shuffle_indices]
y_train = y_train[shuffle_indices]


# 3. Model learning

In [ ]:
# VGG16 model

from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

from tensorflow.keras.utils import to_categorical

### 에러)`logits` and `labels` must have the same shape, received ((None, 2) vs (None, 1)) 해결하기 위해 one-hot encoding
### one-hot encoding : converts categorical labels into a binary matrix
# Convert y_train and y_test to one-hot encoded format
y_train_one_hot = to_categorical(y_train, num_classes=2)
y_test_one_hot = to_categorical(y_test, num_classes=2)

# Load the VGG16 model (without the top classification layer)
vgg16_base = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Freeze the layers in VGG16
for layer in vgg16_base.layers:
    layer.trainable = False

# Create a new model with VGG16 as the base
model = Sequential()
model.add(vgg16_base)
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(2, activation='sigmoid'))  # Change the units to 2 for binary classification

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 8192)              0         
                                                                 
 dense_2 (Dense)             (None, 128)               1048704   
                                                                 
 dense_3 (Dense)             (None, 2)                 258       
                                                                 
Total params: 15,763,650
Trainable params: 1,048,962
Non-trainable params: 14,714,688
_________________________________________________________________


In [ ]:
# Train the model
model.fit(x_train, y_train_one_hot, epochs=10, batch_size=32, validation_split=0.1)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(x_test, y_test_one_hot)
print("Test accuracy:", test_acc)


Epoch 1/10
8/8 [==============================] - 51s 6s/step - loss: 0.7108 - accuracy: 0.7870 - val_loss: 0.5848 - val_accuracy: 0.8077
Epoch 2/10
8/8 [==============================] - 49s 6s/step - loss: 0.5050 - accuracy: 0.8304 - val_loss: 0.5639 - val_accuracy: 0.8077
Epoch 3/10
8/8 [==============================] - 48s 6s/step - loss: 0.4927 - accuracy: 0.8304 - val_loss: 0.5045 - val_accuracy: 0.8077
Epoch 4/10
8/8 [==============================] - 47s 6s/step - loss: 0.4397 - accuracy: 0.8391 - val_loss: 0.5091 - val_accuracy: 0.8077
Epoch 5/10
8/8 [==============================] - 55s 6s/step - loss: 0.4039 - accuracy: 0.8304 - val_loss: 0.5142 - val_accuracy: 0.8077
Epoch 6/10
8/8 [==============================] - 47s 6s/step - loss: 0.3798 - accuracy: 0.8435 - val_loss: 0.5018 - val_accuracy: 0.8077
Epoch 7/10
8/8 [==============================] - 47s 6s/step - loss: 0.3916 - accuracy: 0.8522 - val_loss: 0.5035 - val_accuracy: 0.7692
Epoch 8/10
8/8 [==================

# 4. Prediction